# Embedding Service

In this notebook will be tested the functionality of the embedding service.

This service must do two main steps:

- chunk a raw text
- embed the chunked text into vectors

To do so, both services requires of an [*embedding model*](https://huggingface.co/blog/getting-started-with-embeddings):

- chunking: Uses the embedding model to generate tokens, this is due to tokenization can has multiple approaches.
- embedding: Uses the embedding model to create the vectors that will represent the data of each chunk generated.

In [13]:
from langchain_text_splitters import TokenTextSplitter
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
import numpy as np
import uuid
from loguru import logger
from typing import Union
from datetime import datetime

import sys

sys.path.append("..")

from rag_llm_energy_expert.services.embeddings.config import EMBEDDINGS_CONFIG

In [3]:
embeddings_config = EMBEDDINGS_CONFIG()

The following text will be used:

In [6]:
text = """
Resumen Ejecutivo\n\n\n3\nI. Introducción\nLa Reforma Energética es un paso decidido rumbo a la modernización del sector energético de \nnuestro país, sin privatizar las empresas públicas dedicadas a la producción y al aprovechamien-\nto de los hidrocarburos y de la electricidad. La Reforma Energética, tanto constitucional como a \nnivel legistlación secundarias, surge del estudio y valoración de las distintas iniciativas presenta-\ndas por los partidos políticos representados en el Congreso.\nLa Reforma Energética tiene los siguientes objetivos y premisas fundamentales:\n1.\t\nMantener la propiedad de la Nación sobre los hidrocarburos que se encuentran en el sub-\nsuelo.\n2.\t\nModernizar y fortalecer, sin privatizar, a Petróleos Mexicanos (Pemex) y a la Comisión Fe-\nderal de Electricidad (CFE) como Empresas Productivas del Estado, 100% públicas y 100% \nmexicanas.\n3.\t\nReducir la exposición del país a los riesgos financieros, geológicos y ambientales en las ac-\ntividades de exploración y extracción de petróleo y gas natural.\n4.\t\nPermitir que la Nación ejerza, de manera exclusiva, la planeación y control del Sistema \nEléctrico Nacional, en beneficio de un sistema competitivo que permita reducir los precios \nde la energía eléctrica.\n5.\t\nAtraer mayor inversión al sector energético mexicano para impulsar el desarrollo del país.\n6.\t\nContar con un mayor abasto de energéticos a mejores precios.\n7. \t Garantizar estándares internacionales de eficiencia, calidad y confiabilidad de suministro \nenergético, así como transparencia y rendición de cuentas en las distintas actividades de la \nindustria energética.\n8.\t\nCombatir de manera efectiva la corrupción en el sector energético.\n9.\t\nFortalecer la administración de los ingresos petroleros e impulsar el ahorro de largo plazo en \nbeneficio de las futuras generaciones.\n10.\t Impulsar el desarrollo, con responsabilidad social y ambiental.\nEstos objetivos se verán traducidos en beneficios concretos para los mexicanos:\n
"""

print(text)


Resumen Ejecutivo


3
I. Introducción
La Reforma Energética es un paso decidido rumbo a la modernización del sector energético de 
nuestro país, sin privatizar las empresas públicas dedicadas a la producción y al aprovechamien-
to de los hidrocarburos y de la electricidad. La Reforma Energética, tanto constitucional como a 
nivel legistlación secundarias, surge del estudio y valoración de las distintas iniciativas presenta-
das por los partidos políticos representados en el Congreso.
La Reforma Energética tiene los siguientes objetivos y premisas fundamentales:
1.	
Mantener la propiedad de la Nación sobre los hidrocarburos que se encuentran en el sub-
suelo.
2.	
Modernizar y fortalecer, sin privatizar, a Petróleos Mexicanos (Pemex) y a la Comisión Fe-
deral de Electricidad (CFE) como Empresas Productivas del Estado, 100% públicas y 100% 
mexicanas.
3.	
Reducir la exposición del país a los riesgos financieros, geológicos y ambientales en las ac-
tividades de exploración y extracción de

## Chunking

**Chunking depends of how many tokens an embedding model supports**

There's a [HuggingFace dashboard](https://huggingface.co/spaces/mteb/leaderboard) that compares the performance of different embedding models. Some metrics to focus on are:

- Number of Parameters: 

    A higher value means the model requires more CPU/GPU memory to run

- Embedding Dimension:

    The dimension of the vectors produced

- Max tokens:

    How many tokens the model can process, the higher the better.


For this time, we'll be using the [*nomic-ai/nomic-embed-text-v2-moe*](https://huggingface.co/nomic-ai/nomic-embed-text-v2-moe) model:

- Number of Parameters: 475M
- Embedding Dimension: 768
- Max tokens: 512



[*Langchain*](https://python.langchain.com/api_reference/text_splitters/base/langchain_text_splitters.base.TextSplitter.html#textsplitter) contains many *text splitters*. Most of them uses string characters to split the data. Nevertheless, an *embedding model* uses *tokens* to delimit the amount of text to be embedded into a vector. To fix this, we can use a Langchain [*TokenTextSplitter*](https://python.langchain.com/api_reference/text_splitters/base/langchain_text_splitters.base.TokenTextSplitter.html) class.


The *TokenTextSplitter* object splits the text into tokens using a model tokenizer. We can use a [*HuggingFace Tokenizer*](https://huggingface.co/docs/transformers/en/main_classes/tokenizer) as an imput parameter for the TokenTextSplitter. A *tokenizer* is in charge of preparing the inputs for a model, this is, convert the text into tokens, then the TokenTextSplitter split the tokens into the chunk_size (max tokens that an embedding model can handle), and then the tokenizer decode the tokens to convert them again into text. 

The [*HuggingFace Tokenizer*](https://huggingface.co/docs/transformers/en/main_classes/tokenizer) library contains tokenizers for all the embedding models.

In [4]:
model = SentenceTransformer(embeddings_config.EMBEDDING_MODEL, trust_remote_code=True)

!!!!!!!!!!!!megablocks not available, using torch.matmul instead


In [5]:
# Initialize an AutoTokenizer instance, this will tokenize the data based on 
# the embedding model used
tokenizer = AutoTokenizer.from_pretrained(embeddings_config.EMBEDDING_MODEL)

# Create a TokenTextSplitter instance, this will use the Tokenizer to
# tokenize the text, split it based on the max tokens supported by the
# embedding model, and then convert the tokens into text again, but now splitted.
splitter = TokenTextSplitter.from_huggingface_tokenizer(
    tokenizer,
    chunk_size=model.max_seq_length,
    chunk_overlap = embeddings_config.CHUNK_OVERLAP,
)

In [9]:
# Returns a list of strings, each entry of the list is a chunk
chunks = splitter.split_text(text)

In [10]:
chunks

['\nResumen Ejecutivo\n\n\n3\nI. Introducción\nLa Reforma Energética es un paso decidido rumbo a la modernización del sector energético de \nnuestro país, sin privatizar las empresas públicas dedicadas a la producción y al aprovechamien-\nto de los hidrocarburos y de la electricidad. La Reforma Energética, tanto constitucional como a \nnivel legistlación secundarias, surge del estudio y valoración de las distintas iniciativas presenta-\ndas por los partidos políticos representados en el Congreso.\nLa Reforma Energética tiene los siguientes objetivos y premisas fundamentales:\n1.\t\nMantener la propiedad de la Nación sobre los hidrocarburos que se encuentran en el sub-\nsuelo.\n2.\t\nModernizar y fortalecer, sin privatizar, a Petróleos Mexicanos (Pemex) y a la Comisión Fe-\nderal de Electricidad (CFE) como Empresas Productivas del Estado, 100% públicas y 100% \nmexicanas.\n3.\t\nReducir la exposición del país a los riesgos financieros, geológicos y ambientales en las ac-\ntividades de e

In [11]:
chunks_sizes = [len(chunk) for chunk in chunks]

print(f"Number of chunks: {len(chunks)} \n"
      f"mean chunk size (in characters): {np.mean(chunks_sizes):.4f}\n"
      f"max chunk size (in characters): {max(chunks_sizes)}"
      )

Number of chunks: 2 
mean chunk size (in characters): 995.0000
max chunk size (in characters): 1395


## Embedding

Now that the data is splitted based on the chunk size of each embedding model. We must notice that a vector DB is the place where we will store the chunks to create the semantic search, to do so, each chunk must be embedded into a vector using the embedding model that fits the chunking size.

There are 3 key elements that define a vector in a vector DB:

- ID
- Dimensions
- Payload

The payload contains all the metadata and the text that was embedded in the vector DB. This is because once the text has been encoded, you cannot retrieve the original data from the vector. So it is necessary to store the text as metadata.

In the next sections, we'll be adding metadata to the chunks created.

In [19]:
# Adding info such as title, and the date of chunking
metadata = {
    "upload_date": datetime.now().strftime(r"%Y-%m-%d"),
    "title": "Title of the text",
    "storage_path": "path/where/the/text/was/stored.pdf",
    }

Then, from the list of strings obtained (chunks), it will be created a more structured chunk, to comply with the necessary of each chunk to be indexed in any vector database

In [25]:
# Embedding the chunk text using batch embedding
chunks_embedded = model.encode(chunks)

# Create a list of dictionaries, which each dictionary is a chunk with all the necessary to be
# indexed into a vector DB
final_chunks = [
    {
        "id": str(uuid.uuid4()),
        "vector": chunks_embedded[i],
        "payload": {
            "text": chunk_text,
            "metadata": metadata,
        },
    }
    for i, chunk_text in enumerate(chunks)
]

In [26]:
final_chunks[0]

{'id': '68db496b-c6ba-4142-b349-0e98a39aba10',
 'vector': array([-0.0463006 , -0.02605172, -0.05845752, -0.00452781,  0.02841991,
        -0.00048111, -0.00669492, -0.01073839, -0.04251841,  0.05817271,
        -0.0112    , -0.0070822 , -0.04240644, -0.00893396, -0.00103696,
         0.05194683,  0.0428669 , -0.08145204, -0.03373823,  0.01772164,
         0.00656532,  0.01129695,  0.00243152, -0.05333932,  0.00192475,
        -0.05817797, -0.064895  ,  0.00815712,  0.02508458, -0.00609556,
        -0.00759712, -0.04305067,  0.00135269,  0.03287392, -0.00298151,
         0.01591194,  0.04459121,  0.02645163, -0.00477658,  0.04339189,
         0.02948121, -0.0428456 ,  0.02268298, -0.03799823, -0.03792908,
         0.00976907,  0.00033429,  0.04159464,  0.03556496,  0.00390328,
         0.03510838,  0.01047253, -0.00072827, -0.0170353 , -0.00954884,
         0.02052629, -0.03044282,  0.03399795,  0.03661748,  0.01052659,
         0.00509544, -0.02198467,  0.0137178 , -0.02834373, -0.0123

At this time, this service would return a json which one of its entries is the list of chunks created